In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium 
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import json

In [ ]:
tiger_df = gpd.read_file('../data/tl_2021_47_tract.shp')
tiger_df = tiger_df.to_crs("EPSG:4326")

In [ ]:
type(tiger_df)

In [ ]:
davidson_loc = tiger_df.loc[(tiger_df.COUNTYFP == '037')]

poverty = pd.read_csv('../data/census_pov.csv')

In [ ]:
davidson_loc.head()

In [ ]:
poverty.head()

In [ ]:
poverty['GEO_ID'].str.split('S')

In [ ]:
poverty['GEO_ID'].str.split('S', expand = True)

In [ ]:
pov = poverty['GEO_ID'].str.split('S', expand = True)
poverty[['US_CODE', 'GEOID']] = pov

poverty.head()

In [ ]:
print('davidson loc shape')
print(davidson_loc.shape)
print('===================')
print('poverty shape')
print(poverty.shape)


In [ ]:
davidson_poverty = pd.merge(davidson_loc, poverty, on = "GEOID", how = "inner")

In [ ]:
davidson_poverty = davidson_poverty.rename(columns={"Total Population for whom poverty status os determined- Estimate":"population"})
davidson_poverty.dropna()

In [ ]:
davidson_poverty.plot()

In [ ]:
type(davidson_poverty)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
davidson_poverty.plot(column = 'population', 
              cmap = 'GnBu', 
              edgecolor = 'black', 
              legend = True, 
              ax = ax)
ax.axis('off');

In [ ]:
davidson_poverty.plot(column = 'population', 
cmap = 'GnBu', 
edgecolor = 'black', 
legend = True)

In [ ]:
davidson_loc.info()

In [ ]:
davidson_poverty['population'].max()

In [ ]:
#Bring in crime data
crime = pd.read_csv('./data/zip_crime.csv')
#bring in zips
zipcodes = gpd.read_file('./data/zipcodes.geojson')

#bring in top and bottom zip info
top_bottom = pd.read_csv('./data/zip_info.csv')

In [ ]:
top_bottom.groupby('top_bottom')['store_count'].sum()

In [ ]:
crime.rename(columns={'zip_code':'zip', 'incident_number':'incident_count'}, inplace=True)
crime = crime[['zip', 'incident_count']]
crime = crime.dropna()
crime

In [ ]:
zipcodes = zipcodes.dropna()
zipcodes['zip'] = zipcodes['zip'].astype(int)
zipcodes

In [ ]:
zip_crime = pd.merge(zipcodes, crime, on = "zip", how = "inner")
zip_crime = zip_crime.dropna()
print(type(zip_crime))
zip_crime.shape

In [ ]:
category = []

for index, row in zip_crime.iterrows():
    if row.incident_count >= 36375:
        category.append('High Crime')
    elif row.incident_count < 36375 and row.incident_count > 18188:
        category.append('Med Crime')
    else:
        category.append('Low Crime')
        
zip_crime['crime_level'] = category        
zip_crime

In [ ]:
zip_n_crime_only = zip_crime[['zip','crime_level', 'po_name']].copy()

In [ ]:
zip_n_crime_only = zip_n_crime_only.drop_duplicates(subset=['zip'])
#zip_n_crime_only.to_csv('zip_n_crime_only.csv')

In [ ]:
star_zips = pd.read_csv('./data/star_zips.csv')
star_zips

In [ ]:
star_crime = pd.merge(star_zips,zip_n_crime_only, left_on='zipcodes',right_on='zip')
star_crime.to_csv('star_crime.csv')
star_crime

In [ ]:
star_crime.groupby('po_name')['location'].count()

In [ ]:
star_crime.groupby('crime_level')['location'].count()

In [ ]:
by_zip = pd.DataFrame(star_crime.groupby('zipcodes')['location'].count())
by_zip = by_zip.reset_index()
by_zip.sort_values(by='location', ascending=False)

In [ ]:
#for markers
starbucks = pd.read_csv('./data/starbucks_markers.csv')
starbucks = starbucks.dropna()
other_locs = pd.read_csv('./data/final_locs.csv')
len(other_locs)


In [ ]:

aunts_house = pd.read_csv('./data/Aunty_crib.csv')
aunts_house

In [ ]:
m = folium.Map(location = [36.174465, -86.767960], zoom_start=10, control_scale=True)
m

In [ ]:
geoJson=davidson_loc.to_crs(epsg=4326).to_json()

folium.Choropleth(
    geo_data =geoJson,                  #json
    name ='choropleth',                  
    data = davidson_poverty,                     
    columns = ['TRACTCE', 'population'], #columns to work on
    key_on ='feature.properties.TRACTCE',
    fill_color ='YlOrBr',     #I passed colors Yellow,Green,Blue
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "Total Population Est"
).add_to(m)

#testing 
for i in range(0,len(starbucks)):
   folium.Marker(
      location=[starbucks.iloc[i]['latitude'], starbucks.iloc[i]['longitude']],
      popup=starbucks.iloc[i]['location'],
   ).add_to(m)

folium.LayerControl().add_to(m)
m

In [ ]:
n = folium.Map(location = [36.174465, -86.767960], zoom_start=11, control_scale=True)

folium.Choropleth(
    geo_data =geoJson,                  #json
    name ='choropleth',                  
    data = davidson_poverty,                     
    columns = ['TRACTCE', 'population'], #columns to work on
    key_on ='feature.properties.TRACTCE',
    fill_color ='YlOrBr',     #I passed colors Yellow,Green,Blue
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "Total Population Est"
).add_to(n)

folium.Choropleth(
    geo_data =zipcodes,                  #json
    name ='choropleth',                  
    data = zip_crime,                     
    columns = ['zip', 'incident_count'], #columns to work on
    key_on ='feature.properties.zip',
    fill_color ='Reds',     #I passed colors Yellow,Green,Blue
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "Crime by incident count"
).add_to(n)

for i, r in starbucks.iterrows():
    loc = [r['latitude'],r['longitude']]
    pop = str(r['location'])
    icon=folium.Icon(color="green",icon="fa-coffee", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon).add_to(n)

for i, r in other_locs.iterrows():
    loc = [r['lat'],r['lon']]
    pop = str(r['location'])
    icon=folium.Icon(color="green",icon="fa-coffee", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon).add_to(n)   

folium.Marker(location=[36.23314,-86.76915], popup = 'Aunts House',
             icon=folium.Icon(color='blue', icon="fa-home", prefix='fa')).add_to(n)
folium.CircleMarker(location=(36.23314,-86.76915),radius=15, fill_color='green').add_to(n)
    
folium.LayerControl().add_to(n)
#folium.GeoJson(data=zip_crime['geometry']).add_to(n)
n

In [ ]:
n.save("index.html")

In [ ]:
for _, r in davidson_poverty.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    dav_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = dav_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    geo_j.add_to(m)

    
m